In [2]:
import os
os.environ['KERAS_BACKEND' ] = 'tensorflow'
os.environ['MKL_THREADING_LAYER'] = 'GNU'
import keras as ks
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import TensorBoard
import keras
import pandas as pd
import numpy as np
from keras import optimizers
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout, Activation
from sklearn.preprocessing import MinMaxScaler
import time 


def main():

    # Set seed for reproducibility
    NAME = "MLP"
    np.random.seed(0)
    print("Loading data...")
    # Load the data from the CSV files
    training_data = pd.read_csv('numerai_training_data.csv', header=0)
    print('original train data shape: {},\t{} \n\n \t:'.format(training_data.shape[0],training_data.shape[1]))

    prediction_data = pd.read_csv('numerai_tournament_data.csv', header=0)
    print('original prediction data shape: {},\t{} \n\n \t:'.format(prediction_data.shape[0],prediction_data.shape[1]))
    
    complete_training_data = pd.concat([training_data, prediction_data])
    print('total training / valdation shape {}'.format(complete_training_data))
    
    # Transform the loaded CSV data into numpy arrays

    features = [f for f in list(training_data) if "feature" in f]
    print(features)

    X = training_data[features]
    mini= MinMaxScaler(feature_range=(0,1)) 
    X = mini.fit_transform(X)

    Y = training_data["target_frank"]
    Y= keras.utils.to_categorical(Y,2) 

    x_prediction = prediction_data[features]
    x_prediction = mini.fit_transform(x_prediction)

    ids = prediction_data["id"]  

    batch_size = 710

    dropout = 0.2

    visible = Input(shape=(50,))
    hidden1 = Dense(10, activation='relu')(visible)
    hidden2 = Dense(20, activation='relu')(hidden1)
    hidden3 = Dense(10, activation='relu')(hidden2)
    output = Dense(2, activation='sigmoid')(hidden3)
    model = Model(inputs=visible, outputs=output)
    model.compile(loss='binary_crossentropy',optimizer='rmsprop')
    model.summary()
    tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
    model.fit(X,Y,batch_size=batch_size,epochs=10,validation_split=0.33,callbacks=[tensorboard])
    

    y_prediction = model.predict(x_prediction)
    evaluate = model.evaluate(x_prediction,y_prediction)
    
    probabilities = y_prediction[:, 1]
    print("- probabilities:", probabilities[1:6])

    # We can see the probability does seem to be good at predicting the
    # true target correctly.
    print("- target:", prediction_data['target_frank'][1:6])
    print("- rounded probability:", [np.round(p) for p in probabilities][1:6])

    # But overall the accuracy is very low.
    correct = [
        np.round(x) == y
        for (x, y) in zip(probabilities, prediction_data['target_frank'])
    ]
    print("- accuracy: ", sum(correct) / float(prediction_data.shape[0]))

    tournament_corr = np.corrcoef(prediction_data['target_frank'],
                                  prediction_data['target_elizabeth'])
    print("- frank vs elizabeth corr:", tournament_corr)
    # You can see that target_elizabeth is accurate using the frank model as well.
    correct = [
        np.round(x) == y
        for (x, y) in zip(probabilities, prediction_data['target_elizabeth'])
    ]
    print("- elizabeth using frank:",
          sum(correct) / float(prediction_data.shape[0]))

    # Numerai measures models on logloss instead of accuracy. The lower the logloss the better.
    # Numerai only pays models with logloss < 0.693 on the live portion of the tournament data.)

    print("- validation logloss:",
          model.evaluate(x_prediction,y_prediction))
    
    results = y_prediction[:, 1]
    results_df = pd.DataFrame(data={'probability_frank':results})

    joined = pd.DataFrame(ids).join(results_df)
    pd.DataFrame(joined[:5])


    print("Writing predictions to predictions.csv")
    path = 'predictions_{:},{}'.format(time.strftime("%Y-%m-%d_%Hh%Mm%Ss", time.gmtime()),NAME) + '.csv'
    print()
    print("Writing predictions to " + path.strip())
    joined.to_csv(path,float_format='%.15f', index=False)

if __name__ == '__main__':

    main()

Loading data...
original train data shape: 502732,	60 

 	:
original prediction data shape: 333917,	60 

 	:
total training / valdation shape                       id   era data_type  feature1  feature2  feature3  \
0       n0003126ff2349f6  era1     train   0.54836   0.31077   0.37524   
1       n003d773d29b57ec  era1     train   0.34712   0.40275   0.42747   
2       n0074df2dc6810b6  era1     train   0.50871   0.48639   0.47544   
3       n0090630f530903e  era1     train   0.61363   0.40268   0.53779   
4       n00af19089546fe9  era1     train   0.30704   0.47273   0.54495   
5       n011d2da12b1e735  era1     train   0.52336   0.59136   0.60506   
6       n014149cadeee55d  era1     train   0.30875   0.62510   0.35229   
7       n0148a4dcf539aba  era1     train   0.40632   0.30590   0.43227   
8       n015855690d31908  era1     train   0.48193   0.27060   0.50228   
9       n0169447f4d6a10e  era1     train   0.51191   0.53663   0.42109   
10      n01703ba4eff8fe7  era1     train   0

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_6 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_7 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 22        
Total params: 962
Trainable params: 962
Non-trainable params: 0
_________________________________________________________________
Train on 336830 samples, validate on 165902 samples
Epoch 1/10
336830/336830 [==============================] - 1s 4us/step - loss: 0.6930 - val

In [1]:
import os
os.environ['KERAS_BACKEND' ] = 'tensorflow'
os.environ['MKL_THREADING_LAYER'] = 'GNU'
import keras as ks
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import TensorBoard
import keras
import pandas as pd
import numpy as np
from keras import optimizers
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout, Activation
from sklearn.preprocessing import MinMaxScaler
import time 


def main():

    # Set seed for reproducibility
    NAME = "MLP"
    np.random.seed(0)



    print("Loading data...")

    # Load the data from the CSV files

    training_data = pd.read_csv('numerai_training_data.csv', header=0)
    print('original train data shape: {},\t{} \n\n \t:'.format(training_data.shape[0],training_data.shape[1]))

    prediction_data = pd.read_csv('numerai_tournament_data.csv', header=0)
    print('original prediction data shape: {},\t{} \n\n \t:'.format(prediction_data.shape[0],prediction_data.shape[1]))
    
    complete_training_data = pd.concat([training_data, prediction_data])
    print('total training / valdation shape {}'.format(complete_training_data))
    
    



    # Transform the loaded CSV data into numpy arrays

    features = [f for f in list(training_data) if "feature" in f]
    print(features)

    X = training_data[features]
    mini= MinMaxScaler(feature_range=(0,1)) 
    X = mini.fit_transform(X)
#     X = X.values
    Y = training_data["target_bernie"]
    Y= keras.utils.to_categorical(Y,2) 

    x_prediction = prediction_data[features]
    x_prediction = mini.fit_transform(x_prediction)

    ids = prediction_data["id"]

#     X = X.values

#     Y = to_categorical(Y, num_classes=2)

    

    batch_size = 710

    dropout = 0.2

    

    m_in = Input(shape=(50,))

    m1 = Dense(50,)(m_in)
    m1 = Activation('relu')(m1)
    m1 = BatchNormalization(momentum=.99999,axis=-1)(m1)

    m2 = Dense(100)(m1)
    m2 = Activation('relu')(m2)
    m2 = BatchNormalization(momentum=.999,axis=-1)(m2)
    
    m3 = Dense(25)(m2)
    m3 = Activation('relu')(m3)
    
    m3 = Dense(25)(m3)
    m3 = Dropout(dropout)(m3) 
    m3 = Activation('relu')(m3)
    
    m3 = Dense(25)(m3)
    m3 = Activation('relu')(m3)
    m3 = BatchNormalization(momentum=.99,axis=-1)(m3)
    
    m3 = Dense(100)(m3)
    m3 = Activation('relu')(m3)
    
    m3 = Dense(25)(m3)
    m3 = Activation('relu')(m3) 
    
    m4 = Dense(25)(m3)
    m4 = Activation('relu')(m4) 
    m4 = Dropout(dropout)(m4) 
    m4 = BatchNormalization(momentum=.9,axis=-1)(m4)

    
    
    m5 = Dense(2)(m4)
    m_out = Activation('sigmoid')(m5)

    model = Model(inputs=m_in, outputs=m_out)



    model.compile(loss='binary_crossentropy',optimizer='rmsprop')
    tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
    model.fit(X,Y,batch_size=batch_size,epochs=10,validation_split=0.33,callbacks=[tensorboard])
    
    

    

    y_prediction = model.predict(x_prediction)
    evaluate = model.evaluate(x_prediction,y_prediction)
    
    probabilities = y_prediction[:, 1]
    print("- probabilities:", probabilities[1:6])

    # We can see the probability does seem to be good at predicting the
    # true target correctly.
    print("- target:", prediction_data['target_bernie'][1:6])
    print("- rounded probability:", [np.round(p) for p in probabilities][1:6])

    # But overall the accuracy is very low.
    correct = [
        np.round(x) == y
        for (x, y) in zip(probabilities, prediction_data['target_bernie'])
    ]
    print("- accuracy: ", sum(correct) / float(prediction_data.shape[0]))

    # The targets for each of the tournaments are very correlated.
    tournament_corr = np.corrcoef(prediction_data['target_bernie'],
                                  prediction_data['target_elizabeth'])
    print("- bernie vs elizabeth corr:", tournament_corr)
    # You can see that target_elizabeth is accurate using the bernie model as well.
    correct = [
        np.round(x) == y
        for (x, y) in zip(probabilities, prediction_data['target_elizabeth'])
    ]
    print("- elizabeth using bernie:",
          sum(correct) / float(prediction_data.shape[0]))

    # Numerai measures models on logloss instead of accuracy. The lower the logloss the better.
    # Numerai only pays models with logloss < 0.693 on the live portion of the tournament data.)

    print("- validation logloss:",
          model.evaluate(x_prediction,y_prediction))

    results = y_prediction[:, 1]

    # -----

    

    results_df = pd.DataFrame(data={'probability_bernie':results})

    joined = pd.DataFrame(ids).join(results_df)
    pd.DataFrame(joined[:5])


    print("Writing predictions to predictions.csv")

    # Save the predictions out to a CSV file
    path = 'predictions_{:}'.format(time.strftime("%Y-%m-%d_%Hh%Mm%Ss", time.gmtime())) + '.csv'
    print()
    print("Writing predictions to " + path.strip())
    # # Save the predictions out to a CSV file
    joined.to_csv(path,float_format='%.15f', index=False)


    # Now you can upload these predictions on numer.ai





if __name__ == '__main__':

    main()

Using TensorFlow backend.


Loading data...
original train data shape: 502732,	60 

 	:
original prediction data shape: 333917,	60 

 	:
total training / valdation shape                       id   era data_type  feature1  feature2  feature3  \
0       n0003126ff2349f6  era1     train   0.54836   0.31077   0.37524   
1       n003d773d29b57ec  era1     train   0.34712   0.40275   0.42747   
2       n0074df2dc6810b6  era1     train   0.50871   0.48639   0.47544   
3       n0090630f530903e  era1     train   0.61363   0.40268   0.53779   
4       n00af19089546fe9  era1     train   0.30704   0.47273   0.54495   
5       n011d2da12b1e735  era1     train   0.52336   0.59136   0.60506   
6       n014149cadeee55d  era1     train   0.30875   0.62510   0.35229   
7       n0148a4dcf539aba  era1     train   0.40632   0.30590   0.43227   
8       n015855690d31908  era1     train   0.48193   0.27060   0.50228   
9       n0169447f4d6a10e  era1     train   0.51191   0.53663   0.42109   
10      n01703ba4eff8fe7  era1     train   0

Train on 336830 samples, validate on 165902 samples
Epoch 1/10
336830/336830 [==============================] - 7s 21us/step - loss: 0.7100 - val_loss: 0.6933
Epoch 2/10
336830/336830 [==============================] - 7s 20us/step - loss: 0.6951 - val_loss: 0.6930
Epoch 3/10
336830/336830 [==============================] - 6s 19us/step - loss: 0.6928 - val_loss: 0.6937
Epoch 4/10
336830/336830 [==============================] - 6s 19us/step - loss: 0.6922 - val_loss: 0.6933
Epoch 5/10
336830/336830 [==============================] - 6s 19us/step - loss: 0.6920 - val_loss: 0.6951
Epoch 6/10
336830/336830 [==============================] - 7s 19us/step - loss: 0.6918 - val_loss: 0.6949
Epoch 7/10
336830/336830 [==============================] - 6s 19us/step - loss: 0.6916 - val_loss: 0.6941
Epoch 8/10
336830/336830 [==============================] - 6s 19us/step - loss: 0.6914 - val_loss: 0.6937
Epoch 9/10
336830/336830 [==============================] - 6s 19us/step - loss: 0.6912 - va

In [ ]:
    m4 = Dense(25)(m3)
    m4 = Activation('sigmoid')(m4)
    m4 = Dropout(dropout)(m4) 